## 2.2.1 分词器算法

### BPE（Byte Pair Encoding）

BPE（Byte Pair Encoding）是一种数据压缩算法，后来被广泛应用于自然语言处理中的分词任务。BPE通过逐步合并最常见的字符对来构建词汇表，从而将单词分解为子词单元。这种方法能够有效地处理未登录词（OOV）问题，并且在处理稀有词汇时表现良好。

#### BPE算法的基本步骤

1. **初始化词汇表**：
   - 将每个单词分解为字符序列，并将这些字符作为初始词汇表。

2. **统计字符对频率**：
   - 统计所有相邻字符对的出现频率。

3. **合并最常见字符对**：
   - 找到出现频率最高的字符对，并将其合并为一个新的符号，更新词汇表。

4. **重复合并过程**：
   - 重复步骤2和步骤3，直到达到预定的词汇表大小或合并次数。

#### 示例

假设有以下文本数据：

```
"low lower lowest"
```

#### 初始词汇表

首先，我们将每个单词分解为字符序列，并统计每个字符的出现频率：

```
l, o, w,   l, o, w, e, r,   l, o, w, e, s, t
```

初始词汇表为：`{'l', 'o', 'w', 'e', 'r', 's', 't'}`

#### 第一次合并

统计所有相邻字符对的出现频率：

```
lo: 3
ow: 3
we: 1
er: 1
es: 1
st: 1
```

最常见的字符对是`lo`和`ow`，假设我们选择合并`lo`。合并后，词汇表更新为：`{'lo', 'w', 'e', 'r', 's', 't'}`

更新后的文本表示：

```
"lo w lo w e r lo w e s t"
```

#### 第二次合并

再次统计相邻字符对的出现频率：

```
lo w: 3
w e: 1
e r: 1
e s: 1
s t: 1
```

最常见的字符对是`lo w`，我们将其合并为`low`。词汇表更新为：`{'low', 'e', 'r', 's', 't'}`

更新后的文本表示：

```
"low low e r low e s t"
```

#### 第三次合并

统计相邻字符对的出现频率：

```
low e: 2
e r: 1
e s: 1
s t: 1
```

最常见的字符对是`low e`，我们将其合并为`lowe`。词汇表更新为：`{'low', 'lowe', 'r', 's', 't'}`

更新后的文本表示：

```
"low lowe r lowe s t"
```

#### 第四次合并

统计相邻字符对的出现频率：

```
low r: 1
r lowe: 1
lowe s: 1
s t: 1
```

最常见的字符对是`lowe s`，我们将其合并为`lowes`。词汇表更新为：`{'low', 'lowe', 'lowes', 'r', 't'}`

更新后的文本表示：

```
"low lowe r lowes t"
```

#### 第五次合并

统计相邻字符对的出现频率：

```
low r: 1
r lowes: 1
lowes t: 1
```

最常见的字符对是`lowes t`，我们将其合并为`lowest`。词汇表更新为：`{'low', 'lowe', 'lowes', 'lowest', 'r'}`

更新后的文本表示：

```
"low lowe r lowest"
```

In [ ]:
### BPE算法
from collections import defaultdict, Counter

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, vocab_in):
    vocab_out = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in vocab_in:
        w_out = word.replace(bigram, replacement)
        vocab_out[w_out] = vocab_in[word]
    return vocab_out

def bpe_tokenize(text, num_merges):
    # 初始化词汇表
    vocab = {' '.join(word): freq for word, freq in Counter(text.split()).items()}
    
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best_pair = max(pairs, key=pairs.get)
        vocab = merge_vocab(best_pair, vocab)
    
    return vocab

# 示例文本
text = "low lower lowest good god goods new news"

# 设置合并次数
num_merges = 20

# 执行BPE算法
vocab = bpe_tokenize(text, num_merges)

# 输出最终的词汇表
print("Final Vocabulary:", vocab)

Final Vocabulary: {'low': 1, 'lower': 1, 'lowest': 1, 'good': 1, 'god': 1, 'goods': 1, 'new': 1, 'news': 1}


### WordPiece
WordPiece 是一种广泛应用于自然语言处理中的分词算法，特别是在BERT等预训练语言模型中。WordPiece 与 BPE（Byte Pair Encoding）类似，但有一些关键区别。WordPiece 的目标是通过最大化语言模型的对数似然来选择最佳的子词单元，而不是简单地基于字符对频率进行合并。

#### WordPiece 算法的基本步骤

1. **初始化词汇表**：
   - 将每个单词分解为字符序列，并将这些字符作为初始词汇表。

2. **训练语言模型**：
   - 使用初始词汇表训练一个语言模型，计算每个子词单元的概率。

3. **选择最佳合并**：
   - 选择能够最大化语言模型对数似然的字符对进行合并。

4. **更新词汇表**：
   - 将合并后的新子词单元加入词汇表。

5. **重复合并过程**：
   - 重复步骤2到步骤4，直到达到预定的词汇表大小或合并次数。

#### 示例

假设有以下文本数据：

```
"low lower lowest"
```

#### 初始词汇表

首先，我们将每个单词分解为字符序列，并统计每个字符的出现频率：

```
l, o, w,   l, o, w, e, r,   l, o, w, e, s, t
```

初始词汇表为：`{'l', 'o', 'w', 'e', 'r', 's', 't'}`

#### 第一次合并

假设我们训练了一个语言模型，并计算了每个字符对的对数似然。假设`lo`的对数似然最高，我们选择合并`lo`。合并后，词汇表更新为：`{'lo', 'w', 'e', 'r', 's', 't'}`

更新后的文本表示：

```
"lo w lo w e r lo w e s t"
```

#### 第二次合并

再次训练语言模型，并计算每个字符对的对数似然。假设`low`的对数似然最高，我们选择合并`low`。词汇表更新为：`{'low', 'e', 'r', 's', 't'}`

更新后的文本表示：

```
"low low e r low e s t"
```

#### 第三次合并

再次训练语言模型，并计算每个字符对的对数似然。假设`lowe`的对数似然最高，我们选择合并`lowe`。词汇表更新为：`{'low', 'lowe', 'r', 's', 't'}`

更新后的文本表示：

```
"low lowe r lowe s t"
```

#### 第四次合并

再次训练语言模型，并计算每个字符对的对数似然。假设`lowes`的对数似然最高，我们选择合并`lowes`。词汇表更新为：`{'low', 'lowe', 'lowes', 'r', 't'}`

更新后的文本表示：

```
"low lowe r lowes t"
```

#### 第五次合并

再次训练语言模型，并计算每个字符对的对数似然。假设`lowest`的对数似然最高，我们选择合并`lowest`。词汇表更新为：`{'low', 'lowe', 'lowes', 'lowest', 'r'}`

更新后的文本表示：

```
"low lowe r lowest"
```

In [11]:
### WordPiece算法
from collections import defaultdict, Counter
import math

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, vocab_in):
    vocab_out = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in vocab_in:
        w_out = word.replace(bigram, replacement)
        vocab_out[w_out] = vocab_in[word]
    return vocab_out

def compute_log_likelihood(vocab, pair):
    # 这里简化了计算，实际应用中需要使用语言模型来计算对数似然
    return math.log(sum(vocab.values()))

def wordpiece_tokenize(text, num_merges):
    # 初始化词汇表
    vocab = {' '.join(word): freq for word, freq in Counter(text.split()).items()}
    
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        # 选择能够最大化对数似然的字符对
        best_pair = max(pairs, key=lambda p: compute_log_likelihood(vocab, p))
        vocab = merge_vocab(best_pair, vocab)
    
    return vocab

# 示例文本
text = "low lower lowest"

# 设置合并次数
num_merges = 10

# 执行WordPiece算法
vocab = wordpiece_tokenize(text, num_merges)

# 输出最终的词汇表
print("Final Vocabulary:", vocab)

Final Vocabulary: {'low': 1, 'lower': 1, 'lowest': 1}


### Unigram

Unigram 分词算法是一种基于概率的分词方法，它通过最大化句子的概率来选择最佳的分词方式。与 BPE 和 WordPiece 不同，Unigram 算法不是通过合并字符对来构建词汇表，而是直接通过概率模型来选择最优的子词单元。

#### Unigram 算法的基本步骤

1. **初始化词汇表**：
   - 使用所有可能的字符和子词单元初始化词汇表。

2. **训练语言模型**：
   - 使用 EM 算法（Expectation-Maximization）训练一个语言模型，计算每个子词单元的概率。

3. **选择最佳分词**：
   - 对于每个句子，选择能够最大化句子概率的分词方式。

4. **更新词汇表**：
   - 根据分词结果更新词汇表和子词单元的概率。

5. **重复训练过程**：
   - 重复步骤2到步骤4，直到词汇表和子词单元的概率收敛。

#### 示例

假设我们有以下文本数据：

```
"low lower lowest"
```

#### 初始词汇表

首先，我们将每个单词分解为字符序列，并将这些字符作为初始词汇表：

```
l, o, w, e, r, s, t
```

初始词汇表为：`{'l', 'o', 'w', 'e', 'r', 's', 't'}`

#### 第一次训练

假设我们训练了一个语言模型，并计算了每个子词单元的概率。假设初始概率如下：

```
P(l) = 0.2, P(o) = 0.2, P(w) = 0.2, P(e) = 0.1, P(r) = 0.1, P(s) = 0.1, P(t) = 0.1
```

#### 第一次分词

对于句子 `"low"`，我们有以下可能的分词方式：

1. `l o w`：概率为 `P(l) * P(o) * P(w) = 0.2 * 0.2 * 0.2 = 0.008`
2. `lo w`：假设 `lo` 是一个新的子词单元，概率为 `P(lo) * P(w) = 0.3 * 0.2 = 0.06`
3. `low`：假设 `low` 是一个新的子词单元，概率为 `P(low) = 0.5`

选择概率最大的分词方式 `low`。

#### 更新词汇表

将 `low` 加入词汇表，并更新子词单元的概率：

```
P(low) = 0.5, P(l) = 0.1, P(o) = 0.1, P(w) = 0.1, P(e) = 0.1, P(r) = 0.1, P(s) = 0.1, P(t) = 0.1
```

#### 第二次训练

再次训练语言模型，并计算每个子词单元的概率。假设更新后的概率如下：

```
P(low) = 0.5, P(l) = 0.1, P(o) = 0.1, P(w) = 0.1, P(e) = 0.1, P(r) = 0.1, P(s) = 0.1, P(t) = 0.1
```

#### 第二次分词

对于句子 `"lower"`，我们有以下可能的分词方式：

1. `l o w e r`：概率为 `P(l) * P(o) * P(w) * P(e) * P(r) = 0.1 * 0.1 * 0.1 * 0.1 * 0.1 = 0.00001`
2. `low e r`：概率为 `P(low) * P(e) * P(r) = 0.5 * 0.1 * 0.1 = 0.005`
3. `lo w e r`：概率为 `P(lo) * P(w) * P(e) * P(r) = 0.3 * 0.1 * 0.1 * 0.1 = 0.0003`
4. `lower`：假设 `lower` 是一个新的子词单元，概率为 `P(lower) = 0.6`

选择概率最大的分词方式 `lower`。

#### 更新词汇表

将 `lower` 加入词汇表，并更新子词单元的概率：

```
P(lower) = 0.6, P(low) = 0.3, P(l) = 0.05, P(o) = 0.05, P(w) = 0.05, P(e) = 0.05, P(r) = 0.05, P(s) = 0.05, P(t) = 0.05
```

In [ ]:
### Unigram算法
from collections import defaultdict, Counter
import math

def initialize_vocab(text):
    # 初始化词汇表，包含所有字符
    vocab = set()
    for word in text.split():
        vocab.update(list(word))
    return vocab

def compute_sentence_probability(sentence, vocab, probs):
    # 计算句子的概率
    probability = 1.0
    for token in sentence.split():
        if token in probs:
            probability *= probs[token]
        else:
            probability = 0.0
            break
    return probability

def unigram_tokenize(text, vocab, probs):
    # 对文本进行分词
    tokens = []
    for word in text.split():
        best_split = None
        max_prob = 0.0
        # 尝试所有可能的分词方式
        for i in range(1, len(word) + 1):
            prefix = word[:i]
            suffix = word[i:]
            if prefix in vocab:
                prefix_prob = probs[prefix]
                suffix_prob = compute_sentence_probability(suffix, vocab, probs)
                total_prob = prefix_prob * suffix_prob
                if total_prob > max_prob:
                    max_prob = total_prob
                    best_split = (prefix, suffix)
        if best_split:
            tokens.append(best_split[0])
            if best_split[1]:
                tokens.extend(unigram_tokenize(best_split[1], vocab, probs))
        else:
            tokens.append(word)
    return tokens

def train_unigram(text, num_iterations):
    # 初始化词汇表
    vocab = initialize_vocab(text)
    # 初始化子词单元的概率
    probs = {token: 1.0 / len(vocab) for token in vocab}
    
    for _ in range(num_iterations):
        # 分词
        tokens = unigram_tokenize(text, vocab, probs)
        # 统计子词单元的频率
        token_counts = Counter(tokens)
        total_count = sum(token_counts.values())
        # 更新子词单元的概率
        for token in token_counts:
            probs[token] = token_counts[token] / total_count
        # 更新词汇表
        vocab.update(token_counts.keys())
    
    return vocab, probs

# 示例文本
text = "low lower lowest today good day is sky ok a tree new miss to"

# 设置训练迭代次数
num_iterations = 10

# 训练Unigram模型
vocab, probs = train_unigram(text, num_iterations)

# 输出最终的词汇表和子词单元的概率
print("Final Vocabulary:", vocab)
print("Final Probabilities:", probs)

# 对新的句子进行分词
new_sentence = "Today is a good day"
tokens = unigram_tokenize(new_sentence, vocab, probs)
print("Tokenized Sentence:", tokens)

### SentencePiece

**SentencePiece** 是一种无监督的分词算法，广泛应用于自然语言处理任务中（如机器翻译、文本生成等）。它的核心思想是将文本直接视为一个字符序列，并通过统计学习方法（如 BPE 或 Unigram）将字符序列分割成子词单元。SentencePiece 的一个重要特点是它不需要预先分词，可以直接从原始文本中学习分词模型。

#### SentencePiece 的主要特点

1. **无监督学习**：
   - SentencePiece 直接从原始文本中学习分词模型，不需要预先分词。
   
2. **支持多种分词算法**：
   - 支持 BPE（Byte Pair Encoding）和 Unigram 两种分词算法。

3. **字符级别的处理**：
   - 将文本视为字符序列，能够处理任意语言的文本。

4. **子词单元**：
   - 将文本分割成子词单元，能够有效处理未登录词（OOV）和稀有词汇。


#### SentencePiece 的算法过程

#### 1. 初始化
- 将文本视为字符序列，并将每个字符作为初始词汇表。

#### 2. 训练分词模型
- 使用 BPE 或 Unigram 算法训练分词模型：
  - **BPE**：通过合并高频字符对来构建词汇表。
  - **Unigram**：通过最大化句子的概率来选择最佳的子词单元。

#### 3. 分词
- 使用训练好的模型将文本分割成子词单元。

#### 4. 解码
- 将子词单元转换回原始文本。

SentencePiece 官方提供了 Python 库，可以直接使用。以下是使用 SentencePiece 库的示例代码：

#### 安装 SentencePiece

```bash
pip install sentencepiece
```

In [2]:
import sentencepiece as spm

# 示例文本
text = "low lower lowest"

# 将文本保存到文件中
with open("text.txt", "w", encoding="utf-8") as f:
    f.write(text)

# 训练 SentencePiece 模型
spm.SentencePieceTrainer.train(
    input="text.txt",  # 输入文件
    model_prefix="spm_model",  # 模型前缀
    vocab_size=12,  # 词汇表大小
    model_type="bpe",  # 使用 BPE 算法
    character_coverage=1.0,  # 字符覆盖率
    pad_id=0,  # Padding ID
    unk_id=1,  # Unknown token ID
    bos_id=2,  # Beginning of sentence ID
    eos_id=3  # End of sentence ID
)

# 加载训练好的模型
sp = spm.SentencePieceProcessor()
sp.load("spm_model.model")

# 对文本进行分词
tokens = sp.encode_as_pieces("low lower lowest")
print("Tokenized Sentence:", tokens)

# 将子词单元转换回文本
detokenized_text = sp.decode_pieces(tokens)
print("Detokenized Text:", detokenized_text)

Tokenized Sentence: ['▁', 'l', 'o', 'w', '▁', 'l', 'o', 'w', 'e', 'r', '▁', 'l', 'o', 'w', 'e', 's', 't']
Detokenized Text: low lower lowest


### 典型大模型分词器

|算法|模型|
|----|----|
|BPE       |GPT, GPT-2, GPT-J, GPT-Neo, RoBERTa, BART, LLaMA, ChatGLM-6B, Baichuan||
|WordPiece |BERT, DistilBERT，MobileBERT||
|Unigram   |AlBERT, T5, mBART, XLNet|

### Hugging Face分词器

Hugging Face 的 `tokenizers` 库是一个强大的工具，支持多种分词算法。以下是 `tokenizers` 库支持的主要分词算法及其特点：

1. **BPE (Byte Pair Encoding)**
   - **特点**:
     - 通过迭代地合并最频繁出现的字符对来构建词汇表。
     - 能够处理未登录词（OOV），因为它可以将未知单词分解为已知的子词单元。
     - 适用于多种语言，包括低资源语言。
   - **使用场景**:
     - 许多现代NLP模型如OpenAI的GPT系列、Facebook的MarianMT等都使用BPE。

2. **WordPiece**
   - **特点**:
     - 结合了字节对编码的优点，并保留了一些完整的单词。
     - 使用贪婪算法来构建词汇表。
     - 在保持较小词汇表的同时，尽量保留完整的单词。
   - **使用场景**:
     - Google的BERT模型使用WordPiece作为默认的分词器。
     - 微软的RoBERTa也使用WordPiece。

3. **SentencePiece**
   - **特点**:
     - 支持多种训练算法，包括Unigram、BPE、Char and Word N-gram等。
     - 提供灵活的配置选项，可以根据具体需求选择不同的算法。
     - 高效且易于集成。
   - **使用场景**:
     - Google的T5、Facebook的XLM-R等模型使用SentencePiece。

4. **Unigram**
   - **特点**:
     - 基于统计的方法，根据概率分布选择最优的分割方式。
     - 可以生成更平衡的词汇表。
   - **使用场景**:
     - 适用于需要平衡词汇表的应用。

5. **Metaspace**
   - **特点**:
     - 基于空格和特殊标记的分词方法。
     - 适用于需要简单高效的分词任务。
   - **使用场景**:
     - 一些特定的任务或研究项目。

6. **Char-level Tokenization**
   - **特点**:
     - 将文本分割成单个字符。
     - 适合处理低资源语言或需要精细粒度的场景。
   - **使用场景**:
     - 某些特定任务或研究项目可能会使用字符级别的分词。

7. **Whitespace Tokenization**
   - **特点**:
     - 简单地按照空格将文本分割成单词。
     - 实现简单，但不能处理复合词或未登录词。
   - **使用场景**:
     - 基础NLP任务或原型系统开发。

8. **Pretokenized Input**
   - **特点**:
     - 直接接受预分好的单词列表。
     - 适用于已经进行过分词的数据集。
   - **使用场景**:
     - 数据集已经经过预处理的情况。

In [ ]:
# BPE 分词器
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, normalizers

# 创建BPE模型
tokenizer = Tokenizer(models.BPE())

# 添加预处理器
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 训练BPE模型
files = ["path/to/train.txt"]
tokenizer.train(files)

# 保存模型
tokenizer.save("bpe-tokenizer.json")

# 加载模型
tokenizer = Tokenizer.from_file("bpe-tokenizer.json")

# 对文本进行分词
text = "Hello, how are you doing today?"
output = tokenizer.encode(text)
print("Tokens:", output.tokens)

In [ ]:
# WordPiece 分词器
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, normalizers

# 创建WordPiece模型
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

# 添加预处理器
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 训练WordPiece模型
files = ["path/to/train.txt"]
tokenizer.train(files)

# 保存模型
tokenizer.save("wordpiece-tokenizer.json")

# 加载模型
tokenizer = Tokenizer.from_file("wordpiece-tokenizer.json")

# 对文本进行分词
text = "Hello, how are you doing today?"
output = tokenizer.encode(text)
print("Tokens:", output.tokens)

In [ ]:
#### SentencePiece 分词器
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, normalizers

# 创建SentencePiece模型
tokenizer = Tokenizer(models.Unigram())

# 添加预处理器
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 训练SentencePiece模型
files = ["train.txt"]
tokenizer.train(files)

# 保存模型
tokenizer.save("sentencepiece-tokenizer.json")

# 加载模型
tokenizer = Tokenizer.from_file("sentencepiece-tokenizer.json")

# 对文本进行分词
text = "Hello, how are you doing today?"
output = tokenizer.encode(text)
print("Tokens:", output.tokens)

### **Tiktoken**

**Tiktoken** 是 OpenAI 开发的一个高效的分词工具，专门用于 GPT 系列模型（如 GPT-3、GPT-4）的分词任务。它基于字节对编码（BPE，Byte Pair Encoding）算法，能够将文本快速转换为模型所需的 token ID 序列，同时支持将 token ID 序列转换回原始文本。

Tiktoken 的主要特点包括：
1. **高效性**：针对 GPT 系列模型优化，分词速度极快。
2. **多语言支持**：能够处理多种语言的文本。
3. **可扩展性**：支持自定义词汇表和分词规则。
4. **与 GPT 模型兼容**：直接与 OpenAI 的 GPT 系列模型集成。

Tiktoken 的核心功能包括：
- **编码（Tokenization）**：将文本转换为 token ID 序列。
- **解码（Detokenization）**：将 token ID 序列转换回文本。

### **Tiktoken 的使用场景**

1. **文本预处理**：
   - 在将文本输入 GPT 模型之前，需要将文本转换为 token ID 序列。
2. **文本生成**：
   - 在 GPT 模型生成文本后，需要将 token ID 序列转换回可读的文本。
3. **计算 token 数量**：
   - 统计文本的 token 数量，用于控制输入长度或计算成本。

### **安装 Tiktoken**
```
pip install tiktoken
```

In [ ]:
import tiktoken

# 加载分词器
model_name = "bert/bert-base-uncased"  # 选择合适的模型名称
encoder = tiktoken.get_encoding("cl100k_base")

# 示例文本
text = "Hello, how are you doing today?"

# 编码：将文本转换为 token ID 序列
token_ids = encoder.encode(text)
print("Token IDs:", token_ids)

# 解码：将 token ID 序列转换回文本
decoded_text = encoder.decode(token_ids)
print("Decoded Text:", decoded_text)

# 统计 token 数量
token_count = len(token_ids)
print("Token Count:", token_count)